In [1]:
import requests
from PIL import Image
from io import BytesIO
import numpy as np
import math
from config import mapbox_api_key

In [2]:
MAP_BOX_URL = 'https://api.mapbox.com/v4/mapbox.terrain-rgb/{zoom}/{x}/{y}.pngraw?access_token={token}'

In [3]:
overview = Image.new('RGB', (8192, 8192))
tiles = {}

In [4]:
city = {'nw_tiles': [4714, 6372, 14], 'se_tiles': [4764, 6404]}

In [5]:
xrange = list(range(city['nw_tiles'][0], city['se_tiles'][0]))
yrange = list(range(city['nw_tiles'][1], city['se_tiles'][1]))

In [6]:
# Set up fetch tile function
def fetch_tile(x, y, zoom=city['nw_tiles'][2]):
    url = MAP_BOX_URL.format(x=x, y=y, zoom=zoom, token=mapbox_api_key)
    r = requests.get(url)
    c = r.content
    if c[0] == 123:
        return None
    return Image.open(BytesIO(c))

In [7]:
for x in xrange:
    for y in yrange:
        t = (x, y)
        if t not in tiles:
            tiles[t] = fetch_tile(x, y)
        if tiles[t] is not None:
            overview.paste(tiles[t], ((x - city['nw_tiles'][0]) * 256, (y - city['nw_tiles'][1])* 256))

In [8]:
elevation_arr = np.array(overview).astype(np.float32)

In [9]:
r = elevation_arr[:, :, 0]
g = elevation_arr[:, :, 1]
b = elevation_arr[:, :, 2]

heights = ((r * 256 * 256 + g * 256 + b) * 0.1) - 10000
heights.shape, heights[0][0]

((8192, 8192), 4.2998047)

In [10]:
Image.fromarray(heights.astype(np.uint8))

In [11]:
def height_to_rgb(x):
    if (x > 0) & (x <= 0.4):
        return np.array([255, 0, 0])
    elif x > 0.4:
        return np.array([249, 228, 183])
    else:
        return np.array([102, 178, 255])

In [12]:
h, w = heights.shape
ele_map = np.zeros((h, w, 3), np.uint8)
for x in range(w):
    for y in range(h):
        ele_map[y][x] = height_to_rgb(heights[y][x])

In [13]:
Image.fromarray(ele_map)